## Packages

In [1]:
# !pip install numpy tensorflow scikit-learn plotly pandas

# For the UCR dataset we clone the git repo (if in Colab/Kaggle env)
!git clone https://github.com/iMohammad97/anomaly_detection

!pip install kaleido

Cloning into 'anomaly_detection'...
remote: Enumerating objects: 3828, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 3828 (delta 16), reused 8 (delta 4), pack-reused 3798 (from 1)
Receiving objects: 100% (3828/3828), 202.52 MiB | 33.48 MiB/s, done.
Resolving deltas: 100% (1623/1623), done.
Updating files: 100% (2721/2721), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 21.3 MB/s eta 0:00:00:00:0100:01


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.utils import custom_object_scope
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from sklearn.metrics import auc, precision_recall_curve, roc_auc_score, precision_score
import glob, os, sys
import kaleido
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm, trange
import shutil
import math

## Metrics

In [ ]:
def pointwise_precision(y_true, y_pred):
    """
    Timepoint-wise precision: fraction of detected anomalies that are correct.
    """
    return precision_score(y_true, y_pred, zero_division=0)


def make_event(y_true, y_pred):
    """
    Converts binary sequences (y_true and y_pred) into a list of (start, end) event tuples.
    """
    y_true_starts = np.argwhere(np.diff(y_true.flatten(), prepend=0) == 1).flatten()
    y_true_ends = np.argwhere(np.diff(y_true.flatten(), append=0) == -1).flatten()
    y_true_events = list(zip(y_true_starts, y_true_ends))

    y_pred_starts = np.argwhere(np.diff(y_pred, prepend=0) == 1).flatten()
    y_pred_ends = np.argwhere(np.diff(y_pred, append=0) == -1).flatten()
    y_pred_events = list(zip(y_pred_starts, y_pred_ends))

    return y_true_events, y_pred_events


def event_wise_recall(y_true_events, y_pred_events):
    """
    Event-based recall. We consider an event 'detected' if the predicted event
    overlaps with the true event in any way.
    """
    detected_events = 0
    for true_event in y_true_events:
        true_start, true_end = true_event
        for pred_event in y_pred_events:
            pred_start, pred_end = pred_event
            if pred_end >= true_start and pred_start <= true_end:
                detected_events += 1
                break
    return detected_events / len(y_true_events) if y_true_events else 0


def composite_f_score(y_true, y_pred):
    """
    Combines timepoint precision and event-wise recall into a single F-score.
    """
    prt = pointwise_precision(y_true, y_pred)
    y_true_events, y_pred_events = make_event(y_true, y_pred)
    rece = event_wise_recall(y_true_events, y_pred_events)
    if prt + rece == 0:
        return 0
    return 2 * (prt * rece) / (prt + rece)


def custom_auc_with_perfect_point(y_true, anomaly_scores, threshold_steps=100, plot=False):
    """
    Generates thresholds, computes precision (timepoint) and recall (event-wise)
    pairs, checks for a perfect point, and computes the AUC (area under the curve)
    on the PR plane.
    """
    percentiles = np.linspace(np.min(anomaly_scores), np.max(anomaly_scores) + 1e-7, threshold_steps)
    precision_list = []
    recall_list = []
    perfect_point_found = False

    for threshold in percentiles:
        y_pred = (anomaly_scores >= threshold).astype(int)
        prt = pointwise_precision(y_true, y_pred)

        y_true_events, y_pred_events = make_event(y_true, y_pred)
        rece = event_wise_recall(y_true_events, y_pred_events)

        precision_list.append(prt)
        recall_list.append(rece)

        if prt == 1 and rece == 1:
            perfect_point_found = True
            break

    # Compute AUC (precision vs recall)
    custom_area = auc(recall_list, precision_list)

    if plot:
        plt.figure(figsize=(8, 6))
        plt.plot(recall_list, precision_list, marker='o', label=f"AUC = {custom_area:.4f}")
        plt.title("Precision-Recall Curve")
        plt.xlabel("Recall")
        plt.ylabel("Precision")
        plt.grid(True)
        plt.legend(loc="best")
        plt.show()

    return custom_area, perfect_point_found


def compute_auc_pr(y_true, anomaly_scores):
    """
    Compute AUC-PR for time-series anomaly detection.
    """
    try:
        precision, recall, _ = precision_recall_curve(y_true, anomaly_scores)
        auc_pr = auc(recall, precision)
    except ValueError:
        print("AUC-PR computation failed: Ensure both classes (0 and 1) are present in y_true.")
        auc_pr = np.nan
    return auc_pr


def compute_auc_roc(y_true, anomaly_scores):
    """
    Compute AUC-ROC for time-series anomaly detection.
    """
    try:
        auc_roc = roc_auc_score(y_true, anomaly_scores)
    except ValueError:
        print("AUC-ROC computation failed: Ensure both classes (0 and 1) are present in y_true.")
        auc_roc = np.nan
    return auc_roc

## Utilities

In [ ]:
def create_windows(data, window_size: int, step_size: int = 1):
    """
    Given a 2D array `data` of shape (N, features), create overlapping windows
    of shape (window_size, features). Returns array of shape (M, window_size, features).
    If data is shorter than window_size, returns None.
    """
    N = data.shape[0]
    if N < window_size:
        return None
    windows = []
    for i in range(0, N - window_size + 1, step_size):
        window = data[i:i+window_size]
        windows.append(window)
    return np.stack(windows, axis=0)

def set_seed(seed):
    np.random.seed(seed)
    tf.random.set_seed(seed)


class UCR(Dataset):
    def __init__(self, path: str, window_size: int, step_size: int = 1, train: bool = True, data_id: int = 0):
        self.path = path
        self.window_size, self.step_size = window_size, step_size
        if train:
            self.data = self.create_windows('train', data_id)
            self.labels = torch.zeros_like(self.data)
        else:
            self.data = self.create_windows('test', data_id)
            self.labels = self.create_windows('labels', data_id)

    def create_windows(self, tag: str, data_id: int):
        files = [f for f in os.listdir(self.path) if f.endswith(f'{tag}.npy')]
        windows = []
        for file_path in files:
            if data_id != 0 and not file_path.startswith(f'{str(data_id)}_'):
                continue
            array = np.load(os.path.join(self.path, file_path))
            for i in range(0, len(array) - self.window_size + 1, self.step_size):
                windows.append(array[i:i + self.window_size])
        windows = np.array(windows) # because it's faster
        return torch.tensor(windows, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.labels is not None:
            return self.data[idx], self.labels[idx]
        return self.data[idx]

def get_dataloaders(path: str, window_size: int, batch_size: int, step_size: int = 1, data_id: int = 0, shuffle: bool = False, seed: int = 0):
    torch.manual_seed(seed)
    # Create datasets
    train_dataset = UCR(path, window_size, step_size=step_size, train=True, data_id=data_id)
    test_dataset = UCR(path, window_size, step_size=1, train=False, data_id=data_id) # test step size should always be 1
    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(1)  # Shape: (max_len, 1, d_model)

    def forward(self, x):
        seq_len = x.size(0)
        return x + self.pe[:seq_len, :].to(x.device)